# Load libraries and Data

In [1]:
# standard libraries
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', 30)
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import seaborn as sns
import math
# plt.figure(figsize=(16,6))  # set the plot size
# plt.ylim(-0.01, 0.04)  # set the axis zoom range
# plt.xlim(-0.01, 0.04)

import warnings
warnings.filterwarnings('ignore')

In [2]:
from pyhive import presto
import operator

def get_presto_conn():
    return presto.connect(host='presto-alpha-backend.data.houzz.net', port=8086, username='wandajuan')

def presto_query(query, presto_conn, fetchall=True):
    cursor = presto_conn.cursor()
    cursor.execute(query)
    header = list(map(operator.itemgetter(0), cursor.description))
    return header, cursor.fetchall()

conn = get_presto_conn() # establish the connection

In [3]:
!ls

Homesphere Furniture_20220426201344.xlsx
product lookup.ipynb


In [10]:
df = pd.read_excel('Homesphere Furniture_20220426201344.xlsx', header=1)
df.head()

,Product ID,SKU,Brand ID,Created Date,Country of Manufacture,Product Title,Short Description,Feature-1,Feature-2,Feature-3,Feature-4,Feature-5,Feature-6,Feature-7,Feature-8,Feature-9,Feature-10,Specification 1-Key,Specification 1-Value,Specification 2-Key,Specification 2-Value,Specification 3-Key,Specification 3-Value,Specification 4-Key,Specification 4-Value,Specification 5-Key,Specification 5-Value,Specification 6-Key,Specification 6-Value,Specification 7-Key,Specification 7-Value,Proposition65,Chemicals,For Commercial Use,Color Family,Material,Style,UPC,Active,De-Activation Code,BackOrder Date,Available Quantity,Ships on Pallet,Assembly Required,Shipping Service Level,Lead Time Min,Lead Time Max,Sold in USA,Product Height,Product Width,Product Depth,Product Weight,Package-1 Height,Package-1 Width,Package-1 Depth,Package-1 Weight,Package-1 Quantity,Package-1 Type,Package-1 SKU,Package-2 Height,Package-2 Width,Package-2 Depth,Package-2 Weight,Package-2 Quantity,Package-2 Type,Package-2 SKU,Extra Image,Extra Image.1
0,2508819,HF-5861-2434395,5861,04-19-2022,US,Homesphere Furniture Contemporary Wood 6 Drawer Dresser in White,"Accomplishing an effortless style in your master or guest suite is simple if you start with the right pieces, like this 6-drawer dresser by Homesphere. Its mid-century modern silhouette and contemporary gold metal hairpin legs play up its classic white finish giving you a retro feel for a stylish piece you will love. The six spacious drawers run smoothly on metal glides and give you ample space for all your folded t-shirts, jeans, and extra linens. A wall anchor kit is included to properly secure the dresser to the wall to prevent any tipping injuries. Style this dresser on its own, or combine it with any of the products in the Homesphere collection for a timeless look.",Organize your bedroom and closet with the Homesphere 6 Drawer Dresser,"Made of laminated particleboard and metal legs, the white finish and gold hairpin legs gives the Dresser a mid-century modern style","Store your folded t-shirts, jeans, and pajamas in the 6 spacious drawers that feature durable metal slides for years of use. Display photos and decorations on the top surface",Finish your room with the entire Finley collection for a coordinated look (each sold separately),"The 6 Drawer Dresser ships flat to your door in 2 boxes and 2 adults are recommended to assemble. The top surface can hold 75 lbs. and each drawer will support up to 35 lbs. Assembled dimensions: 36.81""H x 61.65”W x 19.69”D",Wall anchor kit is included to secure the Dresser to the wall and prevent tipping injuries,NaN,NaN,NaN,NaN,Product Dimensions,36.81 H x 61.65 W x 19.69 D,Product Weight,165,Weight Capacity,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,No,White,Engineered Wood,Contemporary,680499403645,No,NaN,NaN,1398,No,Yes,Small Parcel,1,2,Yes,36.81,61.65,19.69,165.00,5.00,20.00,41.75,69.00,1,Carton,5396015COM2,4.12,20.75,67.0,108.0,1.0,Carton,5396015COM1,NaN,NaN
1,2508827,HF-5861-497384,5861,04-19-2022,VN,"Homesphere Furniture Transitional 24"" Wood Bar Stool in Dark Brown","Amplify the style of your home with the addition of this counter stool by Homesphere Furniture. The counter stool features a saddle design seat for comfort and four footrails for stability and durability. The stool has a sturdy wood frame and a dark brown finish making it a great addition to your home. This barstool has a 24"" Seat Height and a 275 pound weight limit.",Crafted from Rubberwood,Dark Brown Finish,Sturdy and Durable,Classic style and design,Four footrails for stability,Counter Height,NaN,NaN,NaN,NaN,Assembly Required,Some Assembly Required,Product Dimensions,"16.38""w x 13.25""d x 24""h",Product Weight,9.92 lbs,Seat Height,"24""",Seat Dimensions,"16.93"" x 9.06"" x 1.10""",Weight Capacity,275 lbs,NaN,NaN,Yes,Di(2-ethylhexyl)phthalate (DEHP);Di(2-ethylhexyl)phthalate (DEHP),No,Chocolate,Wood,Transitional,680499403720,No,NaN,NaN,1088,No,Yes,Small Parcel,0,1,Yes,24.00,16.38,13.25,9.92,5

In [8]:
df['Product ID']

0     2508819
1     2508827
2     2508835
3     2508843
4     2508821
       ...   
45    2508848
46    2508856
47    2508864
48    2508851
49    2508859
Name: Product ID, Length: 50, dtype: int64

In [9]:
df['SKU']

0     HF-5861-2434395
1      HF-5861-497384
2     HF-5861-2307815
3     HF-5861-2326063
4     HF-5861-2169019
           ...       
45    HF-5861-2119899
46    HF-5861-2453144
47    HF-5861-1855453
48    HF-5861-2119900
49    HF-5861-2453064
Name: SKU, Length: 50, dtype: object

In [11]:
df['UPC']

0     680499403645
1     680499403720
2     680499403805
3     680499403881
4     680499403669
          ...     
45    680499403935
46    680499404017
47    680499404093
48    680499403966
49    680499404048
Name: UPC, Length: 50, dtype: int64

In [ ]:
hz_list = ', '.join(list(hz['house_id'].map(str)))

In [28]:
product_codes = "'" + "', '".join(list(df['Product ID'].map(str)))+ "'"
mpns = "'" + "', '".join(list(df['SKU'].map(str))) + "'"
upcs = "'" + "', '".join(list(df['UPC'].map(str))) + "'"

In [23]:
product_codes

"'2508819', '2508827', '2508835', '2508843', '2508821', '2508829', '2508837', '2508845', '2508822', '2508830', '2508838', '2508846', '2508817', '2508825', '2508833', '2508841', '2508823', '2508831', '2508839', '2508847', '2508816', '2508824', '2508832', '2508840', '2508818', '2508826', '2508834', '2508842', '2508820', '2508828', '2508836', '2508844', '2508850', '2508858', '2508854', '2508862', '2508852', '2508860', '2508853', '2508861', '2508855', '2508863', '2508849', '2508857', '2508865', '2508848', '2508856', '2508864', '2508851', '2508859'"

In [19]:
mpns

"'HF-5861-2434395', 'HF-5861-497384', 'HF-5861-2307815', 'HF-5861-2326063', 'HF-5861-2169019', 'HF-5861-1865257', 'HF-5861-1978345', 'HF-5861-423908', 'HF-5861-2412917', 'HF-5861-2243997', 'HF-5861-1985382', 'HF-5861-1453745', 'HF-5861-2314351', 'HF-5861-2001929', 'HF-5861-1730588', 'HF-5861-2123056', 'HF-5861-1974304', 'HF-5861-2013453', 'HF-5861-2495841', 'HF-5861-2123944', 'HF-5861-2314350', 'HF-5861-1455923', 'HF-5861-1730561', 'HF-5861-2123055', 'HF-5861-2314354', 'HF-5861-459657', 'HF-5861-2338816', 'HF-5861-2123319', 'HF-5861-2162528', 'HF-5861-1865256', 'HF-5861-1509158', 'HF-5861-2200583', 'HF-5861-2119902', 'HF-5861-2453042', 'HF-5861-2453036', 'HF-5861-2136130', 'HF-5861-2453162', 'HF-5861-2243870', 'HF-5861-1921386', 'HF-5861-1790409', 'HF-5861-2453153', 'HF-5861-1845332', 'HF-5861-2119901', 'HF-5861-2453145', 'HF-5861-1855454', 'HF-5861-2119899', 'HF-5861-2453144', 'HF-5861-1855453', 'HF-5861-2119900', 'HF-5861-2453064'"

In [29]:
upcs

"'680499403645', '680499403720', '680499403805', '680499403881', '680499403669', '680499403744', '680499403829', '680499403904', '680499403676', '680499403751', '680499403836', '680499403911', '680499403621', '680499403706', '680499403782', '680499403867', '680499403683', '680499403768', '680499403843', '680499403928', '680499403614', '680499403690', '680499403775', '680499403850', '680499403638', '680499403713', '680499403799', '680499403874', '680499403652', '680499403737', '680499403812', '680499403898', '680499403959', '680499404031', '680499403997', '680499404079', '680499403973', '680499404055', '680499403980', '680499404062', '680499404000', '680499404086', '680499403942', '680499404024', '680499404109', '680499403935', '680499404017', '680499404093', '680499403966', '680499404048'"

In [26]:
query = """
select * from shop.vl_pupil
where product_code in ({product_codes})
""".format(product_codes=product_codes)

# query
vl1 = pd.read_sql(query, conn)

In [27]:
vl1

,vendor_listing_id,vendor_id,house_id,title,product_code,availability,quantity,fp,has_promo,final_display_price,trade_display_price,map_override,map_override_percent_value,msrp,dc,dfc,dmap,tc,tfc,tp,dsest_p,vendor_name,seller_type,category_id,l0_category_id,l1_category_id,l2_category_id,l3_category_id,l4_category_id,category_name,l0_category_name,l1_category_name,l2_category_name,l3_category_name,l4_category_name,price,fcth,vl_status,vendor_status,house_status,preferred_vendor_listing_id,rating,num_reviews,ideabook_saves,tm,manufacturer,manufacturer_id,tt,wt,pkwt,lt,ltmx,hz_lt,hz_ltmx,hz_pctl,b,ftcth,tp_id,tpp,all_status_valid,style


In [31]:
query = """
select * from c2.product_attributes
where mpn in ({mpns})
or upc in ({upcs})
""".format(mpns=mpns, upcs=upcs)

# query
pa = pd.read_sql(query, conn)

In [32]:
pa

,house_id,parent_product_id,manufacturer,manufacturer_id,mpn,upc,width,depth,height,weight,materials,designer,color,size,design,configuration,variation_theme,minimum_order_quantity,country_of_origin,attributes,created,modified


# Cannot find products in our catalog based on Product ID, SKU & UPC